<a href="https://colab.research.google.com/github/harnalashok/hadoop/blob/main/spark_dataframe_expts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 12th June, 2021
# Myfolder: github/hadoop
# Objectives:
#             i) Install pyspark on colab
#             ii) Install koalas on colab
#
#
# Java 8 install: https://stackoverflow.com/a/58191107
# Hadoop install: https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
# Spark install:  https://stackoverflow.com/a/64183749
#                 https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

# Full spark install

### 1.0 Libraries

In [17]:
# 1.0 How to set environment variable
import os  
import time  

## 2.0 Define some functions

#### ssh_install()

In [18]:
# 2.0 Function to install ssh client and sshd (Server)
def ssh_install():
  print("\n--1. Download and install ssh server----\n")
  ! sudo apt-get remove openssh-client openssh-server
  ! sudo apt install openssh-client openssh-server
  
  print("\n--2. Restart ssh server----\n")
  ! service ssh restart

#### Java install

In [19]:
# 3.0 Function to download and install java 8
def install_java():
  ! rm -rf /usr/java

  print("\n--Download and install Java 8----\n")
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null        # install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     # set environment variable

  !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  !update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
  
  !mkdir -p /usr/java
  ! ln -s "/usr/lib/jvm/java-8-openjdk-amd64"  "/usr/java"
  ! mv "/usr/java/java-8-openjdk-amd64"  "/usr/java/latest"
  
  !java -version       #check java version
  !javac -version

#### setup ssh passphrase

In [20]:
# 6.0 Function tp setup ssh passphrase
def set_keys():
  print("\n---22. Generate SSH keys----\n")
  ! cd ~ ; pwd 
  ! cd ~ ; ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
  ! cd ~ ; cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
  ! cd ~ ; chmod 0600 ~/.ssh/authorized_keys


#### Set environment

In [21]:
# 7.0 Function to set up environmental variables
def set_env():
  print("\n---23. Set Environment variables----\n")
  # 'export' command does not work in colab
  # https://stackoverflow.com/a/57240319
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"   
  

#### function to install prerequisites
java and ssh<br>


In [22]:
# 8.0 Function to call all functions
def install_components():
  print("\n--Install java----\n")
  ssh_install()
  install_java()  
  #set_keys()
  set_env()


## 3.0 Install components
Start downloading, install and configure. Takes around 2 minutes<br>
Your <u>input *'y'* is required </u>at one place while overwriting earlier ssh keys

In [23]:
# 9.0 Start installation
start = time.time()
install_components()
end = time.time()
print("\n---Time taken----\n")
print((end- start)/60)


--Install java----


--1. Download and install ssh server----

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  ncurses-term python3-certifi python3-chardet python3-idna
  python3-pkg-resources python3-requests python3-six python3-urllib3
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  openssh-client openssh-server openssh-sftp-server ssh-import-id
0 upgraded, 0 newly installed, 4 to remove and 39 not upgraded.
After this operation, 5,235 kB disk space will be freed.
(Reading database ... 164127 files and directories currently installed.)
Removing openssh-server (1:7.6p1-4ubuntu0.3) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of stop.
Removing ssh-import-id (5.7-0ubuntu1.1) ...
Removing openssh-sftp-server (1:7.6p1-4ubuntu0.3) ...
Removing openssh-client (1:7.6p1-4ubuntu0

## 4.0 Install spark
koalas will also be installed

### Define functions

`findspark`: PySpark isn't on `sys.path` by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding `pyspark` to `sys.path` at runtime. `findspark` does the latter.

In [24]:
# 1.0 Function to download and unzip spark
def spark_koalas_install():
  print("\n--1.1 Install findspark----\n")
  !pip install -q findspark

  print("\n--1.2 Install databricks Koalas----\n")
  !pip install koalas

  print("\n--1.3 Download Apache tar.gz----\n")
  ! wget -c https://apachemirror.wuchna.com/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

  print("\n--1.4 Transfer downloaded content and unzip tar.gz----\n")
  !  mv /content/spark*   /opt/
  ! tar -xzf /opt/spark-3.1.2-bin-hadoop3.2.tgz  --directory /opt/

  print("\n--1.5 Check folder for files----\n")
  ! ls -la /opt


In [25]:
# 1.1 Function to set environment
def set_spark_env():
  print("\n---2. Set Environment variables----\n")
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre" 
  os.environ["SPARK_HOME"] = "/opt/spark-3.1.2-bin-hadoop3.2"     
  os.environ["LD_LIBRARY_PATH"] += ":/opt/spark-3.1.2-bin-hadoop3.2/lib/native"
  os.environ["PATH"] += ":/opt/spark-3.1.2-bin-hadoop3.2/bin:/opt/spark-3.1.2-bin-hadoop3.2/sbin"
  print("\n---2.1. Check Environment variables----\n")
  # Check
  ! echo $PATH
  ! echo $LD_LIBRARY_PATH

In [26]:
# 1.2 Function to configure spark 
def spark_conf():
  print("\n---3. Configure spark to access hadoop----\n")
  !mv /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh.template  /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh
  #!echo "HADOOP_CONF_DIR=/opt/hadoop-3.2.2/etc/hadoop/" >> /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh
  print("\n---3.1 Check ----\n")
  #!cat /opt/spark-3.1.1-bin-hadoop3.2/conf/spark-env.sh

### Install spark


In [27]:
# 2.0 Call all the three functions
def install_spark():
  spark_koalas_install()
  set_spark_env()
  spark_conf()


In [28]:
# 2.1 
install_spark()


--1.1 Install findspark----


--1.2 Install databricks Koalas----


--1.3 Download Apache tar.gz----

--2021-06-12 13:45:20--  https://apachemirror.wuchna.com/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving apachemirror.wuchna.com (apachemirror.wuchna.com)... 143.110.177.196
Connecting to apachemirror.wuchna.com (apachemirror.wuchna.com)|143.110.177.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M  10.0MB/s    in 25s     

2021-06-12 13:45:46 (8.84 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]


--1.4 Transfer downloaded content and unzip tar.gz----


--1.5 Check folder for files----

total 223492
drwxr-xr-x  1 root root      4096 Jun 12 13:45 .
drwxr-xr-x  1 root root      4096 Jun 12 13:38 ..
drwxr-xr-x  1 root root      4096 Jun  1 13:35 google
drwxr-xr-x  4 root root      4096 Jun  1 13

# Test spark


Call some libraries

In [ ]:
# 3.0 Just call some libraries to test
import pandas as pd
import numpy as np

# 3.1 Get spark in sys.path
import findspark
findspark.init()

# 3.2 Call other spark libraries
#     Just to test
from pyspark.sql import SparkSession
import databricks.koalas as ks
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

### Understanding SparkSession
For Spark configuration options, see [here](http://spark.apache.org/docs/latest/configuration.html#spark-properties)

In [44]:
# 3.1 Build  spark session
#     with certain configuration options
#     .master => Connect to spark which URL? "local" to run locally, 
#                "local[4]" to run locally with 4 cores,
#                or "spark://master:7077" to run on a Spark standalone cluster.
#
spark = SparkSession. \
                    builder. \
                    master("local[*]"). \ 
                    config("spark.driver.memory", "1g"). \
                    getOrCreate()


In [45]:
# 3.1.1
# Get spark configuration
spark.conf.get("spark.driver.memory")

'1g'

In [46]:
# 3.1.2 
# Get spark session 
abc = spark.builder.getOrCreate()

### Creating spark dataframe

#### From pandas dataframe

In [ ]:
# 4.0 Pandas DataFrame
pdf = pd.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c', 'd','d'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach','apple','orange'],
        'x3': [1, 1, 2, 2, 2, 4, 1, 2],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5, 3.0, 2.0],
        'y1': [1, 0, 1, 0, 0, 1, 1, 0],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes']
    })

# 4.1
pdf

In [ ]:
# 4.2 Transform to Spark DataFrame
#     and print
df = spark.createDataFrame(pdf)
df.show()

In [ ]:
df1 = abc.createDataFrame(pdf)
df1.show()

In [ ]:
############

# Your experiments

In [36]:
# Get existing spark session using builder object
abc = SparkSession.builder.getOrCreate()

In [42]:
abc.conf.get("spark.driver.memory")

'1g'

In [56]:
 df = spark.read.csv(
                     "/content/data_modified.csv",
                     header = True
                     )

In [58]:
df.show(5)

+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave_points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave_points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave_points_worst|symmetry_worst|fractal_dimension_worst|
+---------+-----------+------------+--------------+---------+-----

In [60]:
df1 = df.sample(fraction = 0.5)
df2 = df.sample(fraction = 0.5)

In [61]:
df1.head(4)

[Row(diagnosis='1', radius_mean='17.99', texture_mean='10.38', perimeter_mean='122.8', area_mean='1001', smoothness_mean='0.1184', compactness_mean='0.2776', concavity_mean='0.3001', concave_points_mean='0.1471', symmetry_mean='0.2419', fractal_dimension_mean='0.07871', radius_se='1.095', texture_se='0.9053', perimeter_se='8.589', area_se='153.4', smoothness_se='0.006399', compactness_se='0.04904', concavity_se='0.05373', concave_points_se='0.01587', symmetry_se='0.03003', fractal_dimension_se='0.006193', radius_worst='25.38', texture_worst='17.33', perimeter_worst='184.6', area_worst='2019', smoothness_worst='0.1622', compactness_worst='0.6656', concavity_worst='0.7119', concave_points_worst='0.2654', symmetry_worst='0.4601', fractal_dimension_worst='0.1189'),
 Row(diagnosis='1', radius_mean='11.42', texture_mean='20.38', perimeter_mean='77.58', area_mean='386.1', smoothness_mean='0.1425', compactness_mean='0.2839', concavity_mean='0.2414', concave_points_mean='0.1052', symmetry_mean=

In [ ]:
df1.collect()

In [71]:
len(df1.columns)

31

In [73]:
df1.summary().show()

+-------+-------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------------------+-------------------+------------------+------------------+-----------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+------------------+--------------------+-------------------+-----------------------+
|summary|          diagnosis|       radius_mean|      texture_mean|    perimeter_mean|         area_mean|     smoothness_mean|    compactness_mean|     concavity_mean| concave_points_mean|       symmetry_mean|fractal_dimension_mean|          radius_se|        texture_se|      perimeter_se|          area_se|       smoothness_se|      compactness_se|       concavity_s

In [79]:
print(df1.schema)

StructType(List(StructField(diagnosis,StringType,true),StructField(radius_mean,StringType,true),StructField(texture_mean,StringType,true),StructField(perimeter_mean,StringType,true),StructField(area_mean,StringType,true),StructField(smoothness_mean,StringType,true),StructField(compactness_mean,StringType,true),StructField(concavity_mean,StringType,true),StructField(concave_points_mean,StringType,true),StructField(symmetry_mean,StringType,true),StructField(fractal_dimension_mean,StringType,true),StructField(radius_se,StringType,true),StructField(texture_se,StringType,true),StructField(perimeter_se,StringType,true),StructField(area_se,StringType,true),StructField(smoothness_se,StringType,true),StructField(compactness_se,StringType,true),StructField(concavity_se,StringType,true),StructField(concave_points_se,StringType,true),StructField(symmetry_se,StringType,true),StructField(fractal_dimension_se,StringType,true),StructField(radius_worst,StringType,true),StructField(texture_worst,StringT

In [81]:
df1.describe().show()

+-------+-------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------------------+-------------------+------------------+------------------+-----------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+------------------+--------------------+-------------------+-----------------------+
|summary|          diagnosis|       radius_mean|      texture_mean|    perimeter_mean|         area_mean|     smoothness_mean|    compactness_mean|     concavity_mean| concave_points_mean|       symmetry_mean|fractal_dimension_mean|          radius_se|        texture_se|      perimeter_se|          area_se|       smoothness_se|      compactness_se|       concavity_s

In [82]:
df1.toPandas()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189
1,1,11.42,20.38,77.58,386.1,0.1425,0.2839,0.2414,0.1052,0.2597,0.09744,0.4956,1.156,3.445,27.23,0.00911,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.5,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.173
2,1,12.45,15.7,82.57,477.1,0.1278,0.17,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.00751,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.4,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.1244
3,1,13.71,20.83,90.2,577.9,0.1189,0.1645,0.09366,0.05985,0.2196,0.07451,0.5835,1.377,3.856,50.96,0.008805,0.03029,0.02488,0.01448,0.01486,0.005412,17.06,28.14,110.6,897,0.1654,0.3682,0.2678,0.1556,0.3196,0.1151
4,1,13,21.82,87.5,519.8,0.1273,0.1932,0.1859,0.09353,0.235,0.07389,0.3063,1.002,2.406,24.32,0.005731,0.03502,0.03553,0.01226,0.02143,0.003749,15.49,30.73,106.2,739.3,0.1703,0.5401,0.539,0.206,0.4378,0.1072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,1,21.56,22.39,142,1479,0.111,0.1159,0.2439,0.1389,0.1726,0.05623,1.176,1.256,7.673,158.7,0.0103,0.02891,0.05198,0.02454,0.01114,0.004239,25.45,26.4,166.1,2027,0.141,0.2113,0.4107,0.2216,0.206,0.07115
283,1,20.13,28.25,131.2,1261,0.0978,0.1034,0.144,0.09791,0.1752,0.05533,0.7655,2.463,5.203,99.04,0.005769,0.02423,0.0395,0.01678,0.01898,0.002498,23.69,38.25,155,1731,0.1166,0.1922,0.3215,0.1628,0.2572,0.06637
284,1,16.6,28.08,108.3,858.1,0.08455,0.1023,0.09251,0.05302,0.159,0.05648,0.4564,1.075,3.425,48.55,0.005903,0.03731,0.0473,0.01557,0.01318,0.003892,18.98,34.12,126.7,1124,0.1139,0.3094,0.3403,0.1418,0.2218,0.0782
285,1,20.6,29.33,140.1,1265,0.1178,0.277,0.3514,0.152,0.2397,0.07016,0.726,1.595,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.74,39.42,184.6,1821,0.165,0.8681,0.9387,0.265,0.4087,0.124


In [83]:
df1.count()

287

In [86]:
df1.crosstab("radius_mean", "texture_mean").show()

+------------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+

In [88]:
cols = ["radius_mean", "texture_mean"]
df1.select(*cols)

+-----------+------------+
|radius_mean|texture_mean|
+-----------+------------+
|      17.99|       10.38|
|      11.42|       20.38|
|      12.45|        15.7|
|      13.71|       20.83|
|         13|       21.82|
|      12.46|       24.04|
|      15.85|       23.95|
|      14.54|       27.54|
|      14.68|       20.13|
|      16.13|       20.68|
|      13.54|       14.36|
|      15.34|       14.26|
|      18.61|       20.25|
|      17.57|       15.05|
|      18.63|       25.11|
|      14.25|       21.72|
|      13.03|       18.42|
|      13.48|       20.82|
|      13.44|       21.58|
|      10.95|       21.35|
+-----------+------------+
only showing top 20 rows



In [89]:
df1.selectExpr("radius_mean" , "texture_mean")

AnalysisException: ignored

In [ ]:
df1.